**1.** **Importing the required libraries**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**2.** **Checking if GPU is available for PyTorch**

In [7]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


**3.** **Loading the dataset**

In [8]:
data=load_breast_cancer()
X=data.data
y=data.target

**3.1 Performing exploratory data analysis and feature engineering**

In [9]:
# Finding out the number of rows and columns that we have
X.shape

(569, 30)

In [10]:
# Checking if any of the columns have missing values
print(type(X))

<class 'numpy.ndarray'>


**4. Splitting the data for training and testing**

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [12]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


**5. Scaling the data to ensure normal distribution**

In [13]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

**6. Converting the entire dataset from Numpy to PyTorch Tensors & moving it to GPU**

In [14]:
print(type(X_train))

<class 'numpy.ndarray'>


In [15]:
X_train=torch.tensor(X_train,dtype=torch.float32).to(device)
y_train=torch.tensor(y_train,dtype=torch.float32).to(device)
X_test=torch.tensor(X_test,dtype=torch.float32).to(device)
y_test=torch.tensor(y_test,dtype=torch.float32).to(device)

**7. Defining the Neural Network Architecture in PyTorch**

In [16]:
class NeuralNet(nn.Module):
  def __init__(self, input_size,hidden_size,output_size):
    super(NeuralNet,self).__init__()
    self.fc1=nn.Linear(input_size,hidden_size)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(hidden_size,output_size)
    self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.sigmoid(out)
    return out


**8. Defining the hyperparameters**

In [17]:
input_size=X_train.shape[1]
hidden_size=64
output_size=1
learning_rate=0.01
num_epochs=100

**9. Instantiating the PyTorch Model**

In [18]:
model=NeuralNet(input_size,hidden_size,output_size).to(device)
criterion=nn.BCELoss()
# it is model.parameters() because the name we have given to our object is 'model'
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

**10. Creating the training loop**

In [19]:
for epoch in range(num_epochs):
  model.train()
  # ensure during every iteration the gradients are set to zero before they are calculated again to prevent gradient accumulation
  optimizer.zero_grad()
  # model(X_train) calls the forward() function in PyTorch model directly
  model_output=model(X_train)
  # check the shapes of model_output and why you need to use y_train.view() here
  loss=criterion(model_output,y_train.view(-1,1))
  # calculating the gradients with respect to weights and bias
  loss.backward()
  # taking the gradient descent step
  optimizer.step()


  # calculating accuracy

  with torch.no_grad():
    predicted=model_output.round()
    correct=(predicted==y_train.view(-1,1)).float().sum()
    accuracy=correct/y_train.size(0)

  if (epoch+1)%10 ==0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")



Epoch [10/100], Loss: 0.1385, Accuracy: 95.60%
Epoch [20/100], Loss: 0.0698, Accuracy: 98.24%
Epoch [30/100], Loss: 0.0535, Accuracy: 98.24%
Epoch [40/100], Loss: 0.0414, Accuracy: 98.90%
Epoch [50/100], Loss: 0.0333, Accuracy: 99.12%
Epoch [60/100], Loss: 0.0266, Accuracy: 99.34%
Epoch [70/100], Loss: 0.0212, Accuracy: 99.34%
Epoch [80/100], Loss: 0.0169, Accuracy: 99.56%
Epoch [90/100], Loss: 0.0138, Accuracy: 99.56%
Epoch [100/100], Loss: 0.0113, Accuracy: 99.56%


**11. Evaluating on test dataset**

In [20]:
model.eval()
with torch.no_grad():
  model_output=model(X_test)
  predicted=model_output.round()
  correct=(predicted == y_test.view(-1,1)).float().sum()
  accuracy=correct/y_test.size(0)
  print(f"Accuracy on test dataset: {accuracy.item() * 100:.2f}%")

Accuracy on test dataset: 98.25%
